### Dissertação de Mestrado: Frederico Gomes

• Autor: Maria Luiza Campos

• Data: Julho 2025

**Metodologia:** Teste Exato de Fisher seguido por regressão logística para averiguar robustez. 

In [3]:
## Importando bibliotecas

import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

## Importando os dados

dados = pd.read_csv('dado_limpos.csv')
print("Dados importados com sucesso!")

Dados importados com sucesso!


In [ ]:
## Filtrando apenas para as variáveis de interesse

variaveis_interesse = ['ID_processo','resultado_julgamento', 'categoria_vinculo', 'filhos']
dados_filtrados = dados[variaveis_interesse].copy()
dados_filtrados = dados_filtrados[dados_filtrados['ID_processo'] != 202146] ## removendo um dos processos
print(dados_filtrados)
print(f"Total de observações originais: {len(dados)}")
print(f"Total de observações final: {len(dados_filtrados)}")

     ID_processo  resultado_julgamento  categoria_vinculo  filhos
0         148088                     1                1.0     0.0
1         203354                     1                0.0     0.0
2         211280                     1                1.0     0.0
3         264516                     1                1.0     NaN
4         849912                     1                1.0     1.0
..           ...                   ...                ...     ...
126       197716                     0                1.0     1.0
127       152449                     0                1.0     1.0
128       772844                     0                1.0     0.0
130       210396                     0                NaN     0.0
131       191972                     0                NaN     1.0

[131 rows x 4 columns]
Total de observações originais: 132
Total de observações final: 131


In [16]:
## Verificando missing values

for var in variaveis_interesse:
    missing_count = dados_filtrados[var].isna().sum()
    missing_percent = (missing_count/len(dados_filtrados)) * 100
    print(f"{var}:")
    print(f"  Missing values: {missing_count} ({missing_percent:.1f}%)")
    print(f"  Valores únicos: {dados_filtrados[var].unique()}")

## A porcentagem de valores faltantes é grande, especialmente na existência ou não de filhos. 


ID_processo:
  Missing values: 0 (0.0%)
  Valores únicos: [148088 203354 211280 264516 849912 804741 254352 202422 224010 200221
 184262 132092 835236 173028 191800 193481 694900 184789 174559 146576
 143912 280782 212700 215458 885416 913497 208333 212840 871265 249385
 157443 239068 259005 176898 176979 527774 171531 176559 169452 149635
 175618 140372 143672 155742 158776 231453 134780 195164 831541 800685
 795482 210952 208602 722014 681080 676155 199917 197973 197199 208631
 237957 147619 183064 201216 195723 169551 208489 226153 185259 630372
 188501 189990 142704 163831 153688 379763 477088 382196 387667 160739
 185437 163595 176254 171010 136005 165091 155347 158511 162216 154015
 728575 110419 283470 966382 170792 216534 961744 159217 270098 211850
 214764 231698 248059 279235 166443 172188 118368 221858 230572 201531
 265254 240573 238961 201966 202969 206484 204528 202900 155503 897015
 217780 211854 210161 197716 152449 772844 210396 191972]
resultado_julgamento:
  Missing 

In [17]:
## Vamos remover esses valores faltantes da tabela filtrada

dados_limpos = dados_filtrados.dropna()

for var in variaveis_interesse:
    observacoes_variavel = len(dados_limpos[var])
    print(f"{var}: {observacoes_variavel} observações")

## Isso significa que temos apenas 70 observações com informação COMPLETA. 

ID_processo: 70 observações
resultado_julgamento: 70 observações
categoria_vinculo: 70 observações
filhos: 70 observações


In [31]:
## Vamos rodar o Teste Exato de Fisher para cada uma das variáveis de interesse

## Definindo uma função para o teste

def teste_fisher(var1, var2, dados):
    """
    Executa o teste exato de Fisher entre duas variáveis de interesse

    """
    # Criar tabela de contigência
    tabela = pd.crosstab(dados[var1], dados[var2])

    print(f"\n=== Teste de Fisher: {var1} vs {var2} ===")
    print("Tabela de contigência")
    print(tabela)

    # verificar se a tabela é válida para Fisher
    if tabela.shape != (2, 2):
        print(f"⚠️  Tabela {tabela.shape} - Fisher pode não ser apropriado")
        return None, None, None

    # Executar teste de fisher
    try:
        odds_ratio, p_value = stats.fisher_exact(tabela)

        print(f"Odds Ratio: {odds_ratio:.4f}")
        print(f"P-value: {p_value:.6f}")

         # Interpretação
        if p_value < 0.001:
            sig = "*** (p < 0.001)"
        elif p_value < 0.01:
            sig = "** (p < 0.01)"
        elif p_value < 0.05:
            sig = "* (p < 0.05)"
        else:
            sig = "ns (p >= 0.05)"
        
        print(f"Significância: {sig}")
        
        return odds_ratio, p_value, sig
        
    except Exception as e:
        print(f"❌ Erro no teste: {e}")
        return None, None, None


In [32]:
## Vamos executar o teste

# Inicializar o dicionário de resultados

resultados = {}

## Teste 1: resultado_julgamento vs categoria_vinculo

odds1, p1, sig1 = teste_fisher('resultado_julgamento', 'categoria_vinculo', dados_limpos)
resultados['resultado_julgamento_vs_categoria_vinculo'] = {'odds_ratio': odds1, 'p_value': p1, 'significancia': sig1}

## Teste 2: resultado_julgamento vs filhos 
odds2, p2, sig2 = teste_fisher('resultado_julgamento', 'filhos', dados_limpos)
resultados['resultado_julgamento_vs_filhos'] = {'odds_ratio': odds2, 'p_value': p2, 'significancia': sig2}

for teste, resultado in resultados.items():
    if resultado['odds_ratio'] is not None:
        print(f"\n{teste}:")
        print(f"  Odds Ratio: {resultado['odds_ratio']:.4f}")
        print(f"  P-valor: {resultado['p_value']:.6f}")
        print(f"  Significância: {resultado['significancia']}")
    else:
        print(f"\n{teste}: Teste não executado")


=== Teste de Fisher: resultado_julgamento vs categoria_vinculo ===
Tabela de contigência
categoria_vinculo     0.0  1.0
resultado_julgamento          
0                       0   11
1                      34   25
Odds Ratio: 0.0000
P-value: 0.000410
Significância: *** (p < 0.001)

=== Teste de Fisher: resultado_julgamento vs filhos ===
Tabela de contigência
filhos                0.0  1.0
resultado_julgamento          
0                       3    8
1                      51    8
Odds Ratio: 0.0588
P-value: 0.000155
Significância: *** (p < 0.001)

resultado_julgamento_vs_categoria_vinculo:
  Odds Ratio: 0.0000
  P-valor: 0.000410
  Significância: *** (p < 0.001)

resultado_julgamento_vs_filhos:
  Odds Ratio: 0.0588
  P-valor: 0.000155
  Significância: *** (p < 0.001)


## Interpretação

#### **Resultado do Julgamento vs Tipo de Vínculo**

• Odds Ratio: 0.

• P-valor: 0.000410

1. Isso indica que não houve **nenhum** caso de absolvição quando o vínculo era não afetivo/abuso (razão de chances igual a zero indicando evento impossível de ocorrer). 

2. Já nos casos com vínculo afetivo, houve absolvições (11 absolvições contra 25 condenações).

**Conclusão:** A chance de ser absolvido é significativamente maior quando a relação é afetiva.

**• O p-valor está abaixo de 0.05 (0.000410), indicando uma associação forte entre resultado do julgamento e tipo de vínculo.**

#### **Resultado do Julgamento vs Presença de Filhos**

• Odds Ratio: 0.0588.

• P-valor: 0.000155

1. Ter filhos **diminui** a chance de condenação (Razão de chances menor que um, indicando associação negativa entre a categoria base (1: ser condenado) e a variável (filho entre as partes).)


**Conclusão:** pessoas com filhos têm maior chance de serem absolvidas em comparação com aquelas sem filhos.

**• O p-valor está abaixo de 0.05 (0.000155), indicando uma associação forte entre resultado do julgamento e presença de filhos.**